In [1]:
import os
import pandas as pd
import numpy as np
import values, objects, helpers
import uuid
import random
from datetime import datetime, timedelta

##### Tables

In [2]:
cdhdr_cols = ['MANDT', 'OBJECTCLAS', 'OBJECTID', 'CHANGENR', 'USERNAME', 'ERDAT']
cdpos_cols = ['MANDT', 'CHANGENR', 'TABNAME', 'FNAME', 'OLDVALUE', 'NEWVALUE']
mara_cols = ['MANDT', 'MATNR', 'MAKTX']
usr01_cols = ['MANDT', 'BNAME', 'UTYPE']
Kna1_cols = ['MANDT', 'KUNNR', 'NAME1', 'ERDAT', 'ERNAM']
vbak_cols = ['MANDT', 'VBELN', 'ERDAT', 'ERNAM', 'KUNNR', 'NETWR', 'VSBED', 'FAKSK']
# vbak_cols = ['MANDT', 'VBELN', 'ERDAT', 'ERNAM', 'KUNNR', 'NETWR', 'shp. cond.', 'bil. blk.']
vbap_cols = ['MANDT', 'VBELN', 'POSNR', 'KWMENG', 'MATNR', 'NETWR']
likp_cols = ['MANDT', 'VBELN', 'WADAT', 'LFUHR', 'ERDAT', 'ERNAM']
# likp_cols = ['MANDT', 'id', 'planned. del. date', 'del. date', 'ERDAT', 'ERNAM']
lips_cols = ['MANDT', 'VBELN', 'KDAUF', 'POSNR']
# lips_cols = ['MANDT', 'likp.id', 'vbap.vbeln', 'vbap.posnr']
vbuk_cols = ['MANDT', 'VBELN', 'LFSTK', 'FKSTK', 'AEDAT', 'VBTYP']
# vbuk_cols = ['MANDT', 'VBELN', 'del. stat', 'bil. stat', 'changed on', 'sd cat.']
mkpf_cols = ['MANDT', 'MBLNR', 'BLART', 'AEDAT', 'USNAM']
# mkpf_cols = ['MANDT', 'id', 'BLART', 'AEDAT', 'USNAM']
mseg_cols = ['MANDT', 'MBLNR', 'KDAUF', 'KDPOS']
# mseg_cols = ['MANDT', 'mkpf.id', 'vbap.vbeln', 'vbap.posnr']
vbrk_cols = ['MANDT', 'VBELN', 'VBTYP', 'ERNAM', 'ERDAT']
# vbrk_cols = ['MANDT', 'id', 'VBTYP', 'ERNAM', 'ERDAT']
vbrp_cols = ['MANDT', 'VBELN', 'AUBEL', 'POSNR']
# vbrp_cols = ['MANDT', 'id', 'vbak.vbeln', 'vbap.posnr']

In [3]:
vbak_len = 100

In [4]:
# Materials
mara_temp_list = []
MATERIALS = []
for mat in values.material_names:
    rand_id = uuid.uuid4()
    mara_temp_list.append([values.mandt, rand_id, mat['name']])
    MATERIALS.append(objects.Materials(id=rand_id, name=mat['name'], availability=mat['availability'], price=mat['price']))

mara =  pd.DataFrame(mara_temp_list, columns=mara_cols)

In [5]:
# Users
usr01_temp_list = []
USERS = []
for u_name in values.users:
    usr01_temp_list.append([values.mandt, u_name, 'A'])
    USERS.append(objects.Users(u_name, 'A'))
usr01_temp_list.append([values.mandt, 'BATCH_JOB', 'B'])
USERS.append(objects.Users('BATCH_JOB', 'B'))

usr01 = pd.DataFrame(usr01_temp_list, columns=usr01_cols)

In [6]:
# Customers
kna1_temp_list = []
COMPANIES = []
for company in values.company_names:
    rand_id = uuid.uuid4()
    rand_dt = values.times['customer_create_start'] + helpers.UPTO_YEAR()
    rand_usr = random.choice(list(values.users))
    kna1_temp_list.append([values.mandt, rand_id, company['name'], rand_dt, rand_usr])
    COMPANIES.append(objects.Customer(rand_id, rand_dt, company['credit_worthyness']))

kna1 = pd.DataFrame(kna1_temp_list, columns=Kna1_cols)

In [7]:
vbak_temp_list = []
vbap_temp_list = []
likp_temp_list = []
lips_temp_list = []
cdhdr_changes_temp_list = []
cdpos_changes_temp_list = []
vbuk_temp_list = []
mkpf_temp_list = []
mseg_temp_list = []
vbrk_temp_list = []
vbrp_temp_list = []
transition_matrix = np.array([
    #  [gdd., rdel, sbb., shig, rbb., sniv, cano, rdcn, retg, cinv, end.],
       [0.00, 0.90, 0.05, 0.02, 0.01, 0.00, 0.00, 0.00, 0.00, 0.02, 0.00], # generate delivery doc
       [0.00, 0.00, 0.03, 0.90, 0.00, 0.06, 0.00, 0.00, 0.00, 0.00, 0.00], # release delivery
       [0.50, 0.40, 0.00, 0.05, 0.05, 0.00, 0.10, 0.00, 0.00, 0.00, 0.00], # set billing block
       [0.00, 0.00, 0.00, 0.05, 0.05, 0.90, 0.03, 0.03, 0.01, 0.05, 0.00], # ship goods
       [0.00, 0.00, 0.00, 0.00, 0.05, 0.85, 0.05, 0.01, 0.03, 0.01, 0.00], # remove billing block
       [0.00, 0.00, 0.00, 0.00, 0.00, 0.05, 0.05, 0.90, 0.05, 0.05, 0.00], # send invoice
       [0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 1.00], # cancel order
       [0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.05, 0.10, 0.90, 0.05], # receive delivery confirmation
       [0.00, 0.00, 0.00, 0.00, 0.00, 0.02, 0.90, 0.00, 0.00, 0.00, 0.08], # return goods
       [0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.05, 0.95], # clear invoice
       [0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 1.00]  # end
])
transition_matrix = transition_matrix / transition_matrix.sum(axis=1, keepdims=True)

for i in range(10_000):
    vbak_obj = objects.SalesOrder(users=USERS, companies=COMPANIES, materials=MATERIALS)
    billing_dev_obj = objects.BillingDeviation(vbak=vbak_obj)
    cancel_return_dev_obj = objects.CancelOrReturn(vbak=vbak_obj)

    activities = [
        vbak_obj.generate_delivery_doc, 
        vbak_obj.release_delivery, 
        billing_dev_obj.set_billing_block, 
        vbak_obj.ship_goods, 
        billing_dev_obj.remove_billing_block, 
        vbak_obj.send_invoice, 
        cancel_return_dev_obj.cancel_order, 
        vbak_obj.receive_delivery_confirmation, 
        cancel_return_dev_obj.return_goods, 
        vbak_obj.clear_invoice,
        'end'
    ]

    step = 0
    while step < len(activities) - 1:
        # take a step
        activities[step]()
                
        # step = next_step
        step = np.random.choice(len(activities), p=transition_matrix[step]) # select another step

        # break if end
        if step == len(activities) - 1:
                break

    vbuk_temp_list.append(vbak_obj.vbuk)
    vbak_temp_list.append([values.mandt, vbak_obj.vbeln, vbak_obj.erdat, vbak_obj.ernam, vbak_obj.kunnr, vbak_obj.netwr, vbak_obj.vsbed, vbak_obj.faksk])
    vbap_temp_list += vbak_obj.vbap_temp_list
    likp_temp_list += vbak_obj.likp_temp_list
    lips_temp_list += vbak_obj.lips_temp_list
    cdhdr_changes_temp_list += vbak_obj.cdhdr_changes_temp_list
    cdpos_changes_temp_list += vbak_obj.cdpos_changes_temp_list
    mkpf_temp_list += vbak_obj.mkpf_temp_list
    mseg_temp_list += vbak_obj.mseg_temp_list
    vbrk_temp_list += vbak_obj.vbrk_temp_list
    vbrp_temp_list += vbak_obj.vbrp_temp_list

In [8]:
vbak = pd.DataFrame(vbak_temp_list, columns=vbak_cols)
vbap = pd.DataFrame(vbap_temp_list, columns=vbap_cols)
likp = pd.DataFrame(likp_temp_list, columns=likp_cols)
lips = pd.DataFrame(lips_temp_list, columns=lips_cols)
cdhdr = pd.DataFrame(cdhdr_changes_temp_list, columns=cdhdr_cols)
cdpos = pd.DataFrame(cdpos_changes_temp_list, columns=cdpos_cols)
vbuk = pd.DataFrame(vbuk_temp_list, columns=vbuk_cols)
mkpf = pd.DataFrame(mkpf_temp_list, columns=mkpf_cols)
mseg = pd.DataFrame(mseg_temp_list, columns=mseg_cols)
vbrk = pd.DataFrame(vbrk_temp_list, columns=vbrk_cols)
vbrp = pd.DataFrame(vbrp_temp_list, columns=vbrp_cols)

##### Save csv

In [9]:
version = str(datetime.now())
os.mkdir(f'data/{version}')
cdhdr.to_csv(f'data/{version}/cdhdr.csv', index=False) 
cdpos.to_csv(f'data/{version}/cdpos.csv', index=False) 
mara.to_csv(f'data/{version}/mara.csv', index=False) 
usr01.to_csv(f'data/{version}/usr01.csv', index=False) 
kna1.to_csv(f'data/{version}/kna1.csv', index=False) 
vbak.to_csv(f'data/{version}/vbak.csv', index=False) 
vbap.to_csv(f'data/{version}/vbap.csv', index=False) 
likp.to_csv(f'data/{version}/likp.csv', index=False) 
lips.to_csv(f'data/{version}/lips.csv', index=False) 
vbuk.to_csv(f'data/{version}/vbuk.csv', index=False) 
mkpf.to_csv(f'data/{version}/mkpf.csv', index=False) 
mseg.to_csv(f'data/{version}/mseg.csv', index=False) 
vbrk .to_csv(f'data/{version}/vbrk.csv', index=False) 
vbrp .to_csv(f'data/{version}/vbrp.csv', index=False) 

In [10]:
vbap

,MANDT,VBELN,POSNR,KWMENG,MATNR,NETWR
0,SC1,597900ef-53ca-42f9-8a03-78dee08b670f,0,52,c8f01a8a-4975-459a-85d2-81ad2ba5cbda,171.60
1,SC1,597900ef-53ca-42f9-8a03-78dee08b670f,1,109,8985ce62-e30b-4b48-9798-632a9863c7b4,599.50
2,SC1,597900ef-53ca-42f9-8a03-78dee08b670f,2,30,afbbd857-d468-4c52-b1a8-6368f4f690d5,33.00
3,SC1,597900ef-53ca-42f9-8a03-78dee08b670f,3,61,a176a63f-accd-4c81-af1a-d2843be31cfa,506.30
4,SC1,597900ef-53ca-42f9-8a03-78dee08b670f,4,62,d1dbbe2a-195b-4d9b-96d1-6693b4518630,148.80
...,...,...,...,...,...,...
99902,SC1,25ea2b01-1871-4b04-805c-884c9b89e686,10,73,64c4d916-ca8c-4a0b-8199-ec647e10b0ed,182.50
99903,SC1,25ea2b01-1871-4b04-805c-884c9b89e686,11,64,1b8f2d70-7353-46cb-b77d-3887eb424520,576.00
99904,SC1,25ea2b01-1871-4b04-805c-884c9b89e686,12,115,c8f01a8a-4975-459a-85d2-81ad2ba5cbda,379.50
99905,SC1,25ea2b01-1871-4b04-805c-884c9b89e686,13,94,6e002298-39ed-4ea4-b711-ad2e75cc4f03,752.00
